In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#initializing all the parameters
input_width = 28 
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 5
max_pool2_k = 5

n_hidden = 1024
n_out = 10

input_size_to_hidden = ((input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))  * n_conv2)

In [4]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])), # k*k*c*32
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])), # k*k*32*64
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out]))

}

In [5]:
def conv(x, weights, biases, strides = 1): #apply filter, add biases and then apply activation
    out = tf.nn.conv2d(x, weights, padding='SAME', strides=[1, strides, strides, 1])
    out = tf.nn.bias_add(out, biases)
    out = tf.nn.relu(out)
    return out

def max_pooling(x, k=2):
    return tf.nn.max_pool(x, padding='SAME', ksize=[1,k,k,1], strides = [1,k,k,1]) #need to pass the size of the window and value of stride, here it is k so we move the window by k


In [6]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape=[-1,input_height, input_width, input_channels]) #converting flattend array into square images
    
    conv_1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = max_pooling(conv_1, max_pool1_k) #max_pool1_k is the size of the window

    conv_2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = max_pooling(conv_2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape=[-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1']) #dense layer
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [7]:
x = tf.placeholder("float", [None, input_pixels]) #None as we are not sure about the no. images
y = tf.placeholder(tf.int32, [None, n_out]) #classes
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y)) #using cross entropy cost , the formual for cost we saw in logistic regression


In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size) #by default provides next batch
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y, keep_prob: 0.8}) #optimizing and calc. cost, each run gives low cost
        total_cost += c
    print(total_cost)

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[400,10] labels_size=[100,10]
	 [[Node: softmax_cross_entropy_with_logits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits/Reshape, softmax_cross_entropy_with_logits/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits', defined at:
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/site-

In [ ]:
predictions = tf.argmax(pred, 1) #argmax for index of max value, for this case 1 in one hot encode , parameter 1 tells the axis
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels) #tells us where the value are equal
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                               y:mnist.test.labels, keep_prob: 1.0}) #1.0 means no dropout
correct_predictions.sum()